import pandas as pd
import numpy as np

In [16]:
header = ['user_id','item_id','rating','timestamp']
df = pd.read_csv('/Users/psrini1/Documents/Python/Recommendation system/ml-100k/u.data', sep ='\t', names = header)
## Extracting specific columns from the dataframe
select = ["user_id","item_id"]
df[select]

## Getting the unique count of users / items 

users = df.user_id.unique().shape[0]
users ## 943unique users
items = df.item_id.unique().shape[0]
items ## 1682 unique movies

## Splitting the data using CV 
from sklearn import cross_validation as cv
train_data,test_data = cv.train_test_split(df,test_size = 0.25)

## Creating the user-item matrix 
train_data_matrix = np.zeros((users,items))
train_data_matrix

test_data_matrix = np.zeros((users,items))

## Check what's stored in line
##for line in train_data.itertuples():
  ##  print(line)

for line in train_data.itertuples():
    train_data_matrix [line[1]-1,line[2]-1]= line[3]

for line in test_data.itertuples():
    test_data_matrix[line[1]-1,line[2]-1] =line[3]

In [17]:
## Calculating item and user based similarity scores 

from sklearn.metrics.pairwise import pairwise_distances  

user_similarity = pairwise_distances(train_data_matrix,metric ='cosine')

item_similarity = pairwise_distances(train_data_matrix.T,metric = 'cosine')

In [18]:
a = np.abs(item_similarity).sum(axis=1)
a

array([ 1450.77991186,  1464.05507047,  1530.62747493, ...,  1681.        ,
        1681.        ,  1654.48347936])

In [19]:
a = np.ndarray(shape=(2,2),dtype = int, order = 'F')
b = np.abs(a)
b

array([[0, 2],
       [1, 3]])

In [20]:
## User-user CF

## Averge rating for each user
def prediction(rating,similarity,type):
    if type == 'user':
        mean_rating = rating.mean(axis=1)
        rating_diff = rating-mean_rating[:,np.newaxis]
        pred = mean_rating[:,np.newaxis] + similarity.dot(rating_diff)/ np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = rating.dot(similarity)/np.array([np.abs(similarity).sum(axis=1)])
    return pred


In [21]:
item_prediction = prediction(train_data_matrix,item_similarity,type = 'item')
user_prediction = prediction(train_data_matrix,user_similarity,type ='user')

In [22]:
user_prediction.shape

(943, 1682)

In [28]:
c1 = test_data["user_id"] == 1
c2 = test_data["item_id"] == 1

##train_data[train_data["user_id"] == 38 & train_data["item_id"] == 758]
test_data[c1 & c2]
test_data_matrix[0,0]

0.0

In [25]:
test_data_matrix[test_data_matrix.nonzero()].flatten()

array([ 5.,  4.,  1., ...,  4.,  4.,  3.])

In [26]:
user_prediction[test_data_matrix.nonzero()].flatten()

array([ 0.33933179,  1.36562768,  0.91806102, ...,  0.57697274,
        0.17352656,  0.18445515])

In [32]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse_eval1(prediction,actual):
    prediction = prediction[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction,actual))

rmse_eval1(user_prediction,test_data_matrix)

3.1245470214952173